# Report


---


## 요약

한국어가 가능한 Gemma 2b 모델로 코드 리뷰 과제를 수행했다. RAG 를 사용한 결과 꽤 의미있는 수준의 코드 리뷰를 얻을 수 있었다. 하지만 한 답변 안에서 같은 문장을 반복하는 현상이 몇번 확인되었다. 이후 성능을 향상시키기 위해 다양한 학습을 진행했다. Auto-Regressive 학습, Seq2Seq 학습, GAN 학습이 그것이다. 하지만 추가 학습에서는 별다른 성과를 얻지 못했다.

## 1. 모델 선정

모델의 경우 Gemma 2b 모델 중 한국어가 가능한 모델로 택했다. 약 10GB 의 VRAM 이 있다면 로컬에서도 작동이 가능한 것으로 보였다.

## 2. RAG

DuckDuckGo 검색 엔진 API 의 경우 무료로 검색 기능을 제공한다는 것을 알게되었다. 때문에 이를 Gemma 모델에 결합하여 RAG 로 사용하였다.

원래는 DuckDuckGo 에 검색할 때 질문을 그대로 검색하였다. 하지만 질문이 길거나 복잡할 경우, 검색 결과가 없다고 표시되었다. 때문에 코드 리뷰 데이터에 있는 대부분의 질문에 대해 검색 결과가 없었다.

이를 해결하기 위해 하나의 Gemma 모델을 총 두번 사용하는 쪽으로 변경했다. Gemma 에게 두번에 걸쳐 요청을 보낸다. 첫번째 요청은 코드를 보고, 코드와 관련하여 검색하기 좋은 키워드들을 추려낸다. 재귀 함수, 직렬화, 병렬 연산 등의 키워드가 자주 추려졌다. 두번째 요청에서는 추려진 키워드로 검색을 진행하고, 검색 결과와 함께 코드 리뷰를 진행한다. 그 결과 꽤 의미 있는 수준의 코드 리뷰가 작성된 것으로 보였다.

지금 이 ipynb 파일에서 추가 학습을 진행하기 전 출력 결과가 이에 해당한다. 출력 결과를 보면 같은 문장을 반복해서 생성한 모습들이 일부 보인다.

## 3. 추가 학습 데이터 생성

해당 Gemma 의 성능을 더 끌어올리기 위해 추가 학습을 진행해보기로 하였다.

코드 - 코드 리뷰 데이터 쌍이 필요했다. 해당 데이터의 경우 GPT-4o 를 사용해 생성하는 것이 가장 적합하다 생각하여 그렇게 진행했다. GPT 를 사용해 약 1000개의 코드 - 코드 리뷰 데이터 쌍을 확보했다.

## 4. 추가 학습 Seq2Seq VS 	Auto-Regressive(실패)

Gemma 의 경우 Auto-Regressive 하게 훈련된 모델에 해당한다. 실제로 Gemma 의 사전 학습 모델의 출력을 볼 경우 입력이 출력에 섞여있는 것을 볼 수 있었다. 때문에 처음 Gemma 를 학습 시킬 때는 Auto-Regressive 한 구조로 학습시켰다. 지금 이 ipynb 가 그러하다. 하지만 약 1000개의 데이터로 10 Epoch 간 학습한 결과, Loss 가 거의 줄지 않았다. 추가 학습 전후의 답변을 출력해봐도 크게 다르지 않았다.

때문에 혹시나 하는 마음에 Seq2Seq 방식으로도 학습시켜보기로 했다. 지금 이 ipynb 의 tokenize_batch_seq2seq 함수를 이용했다고 보면 된다. 그 결과 Loss 는 눈에 띄게 줄어갔다. 하지만 결과를 출력해보니, 문장을 전혀 구사하지 못하게 망가진 모습이었다. 이는 Auto-Regressive 모델을 Seq2Seq 하게 훈련시켜서 생긴 오류로 보인다.

결국 두 종류의 추가 학습 모두 눈에 띄는 성능 향상을 이루지 못했다. 데이터 양이 충분하지 않고, 사전 학습 모델이 이미 잘 학습되어있기 때문으로 보인다.

## 5. GAN 과의 결합(실패)

https://github.com/raccoonD0G/AIPlus99/blob/main/Homework4/Homework4_2.py

이전 과제에서 이미지 생성 모델인 GAN 을 사용해보았다. 이 GAN 을 이번 과제에도 적용시켜 보고자 하였다.

GAN 은 G 와 D 가 경쟁적으로 학습한다. 여기서 G 와 D 를 각각 다른 Gemma 모델로 한다. D 는 G가 생성한 답변과 GPT 가 생성한 답변을 입력으로 받으며, 어떤 답변이 진짜인지(GPT 가 생성한 답변인지) 판단한다. G 는 자신이 생성한 답변이 GPT 가 생성한 답변이라고 D 를 속이는 방향으로 학습한다. 결과적으로 G 가 GPT 수준의 답변을 하길 바라는 학습 방향이다. GAN 으로 GPT-4o 모델을 Distillation 하는 것이다.

이 방법 역시 실패했다. 실패 사유는 일단 Gemma 를 두 모델이나 사용해야 해서 VRAM 사용이 지나치게 크고 속도가 느리다. 때문에 한 Epoch 에 수 시간을 소요해 제대로된 테스트가 불가능 했다.

## 6. 결론

이미 사전 학습된 Gemma 2b 는 추가 학습 없이 RAG 만으로도 어느 정도의 성능을 내는것으로 보인다. 하지만 한 답변 안에서 같은 문장을 반복해서 말하는 현상을 몇번 발견했다. 해당 현상을 해결하기 위해 다음주에는 no_repeat_ngram_size, repetition_penalty, temperature 등의 파라미터 값을 조절하는 실험을 진행할 예정이다.


In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["WANDB_DISABLED"] = "true"

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from langchain_community.tools import DuckDuckGoSearchRun
from datasets import load_dataset
import json


CACHE_FILE = "search_cache.json"

# 캐시 불러오기
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r", encoding="utf-8") as f:
        rag_cache = json.load(f)
else:
    rag_cache = {}

search_tool = DuckDuckGoSearchRun()

def get_cached_or_search(question: str) -> str:
    search_query = summarize_for_search_with_gemma(question)

    if search_query in rag_cache:
        return rag_cache[search_query]

    try:
        result = search_tool.run(search_query)
    except Exception as e:
        result = f"검색 실패: {e}"

    rag_cache[search_query] = result

    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(rag_cache, f, ensure_ascii=False, indent=2)

    return result

def summarize_for_search_with_gemma(question: str) -> str:
    prompt = f"""
반드시 한국어로 대답해! 너는 질문을 웹 검색에 적합하도록 간단하고 명확하게 요약해주는 요약기야.
긴 코드 블록이나 설명이 있어도, 요지를 뽑아서 간결한 자연어 검색어로 바꿔줘.
반드시 64 토큰 이내로 요약해야 해 반드시!

[질문]
{question}

[검색용 요약]
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "[검색용 요약]" in summary:
        summary = summary.split("[검색용 요약]")[-1]

    return summary.strip()


def evaluate_on_test_samples(model, tokenizer, test_dataset, num_samples=3):
    print("\n--- 테스트 데이터셋 일부 평가 ---\n")
    samples = test_dataset.select(range(num_samples))

    for i, sample in enumerate(samples):
        question = sample["question"]
        answer = sample["real_answer"]
        context = get_cached_or_search(question)

        prompt = f"""
반드시 한국어로 대답해! 너는 코드 리뷰 전문가야. 아래 검색 결과와 코드 조각을 참고해서 코드에 어떤 문제가 있고 어떻게 개선할 수 있는지 조언을 줘.
추측하지 말고, 검색 결과와 문맥 안의 정보만 사용해. 실제 코드 리뷰처럼 구체적이고 구조적인 피드백을 줘.

[검색 결과]
{context}

[질문]
{question}

[답변]
""".strip()

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=4096)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "[답변]" in response:
            response = response.split("[답변]")[-1]

        print(f"[{i+1}] 질문: {question}\n")
        print(f"정답:\n{answer}\n")
        print(f"모델 응답:\n{response.strip()}\n")
        print("="*80)


# 1. 모델 및 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-2b-it")

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForCausalLM.from_pretrained("unsloth/gemma-2-2b-it", attn_implementation="eager").to("cuda")

model.gradient_checkpointing_enable()

# model 일부 레이어만 학습
for param in model.parameters():
    param.requires_grad = False

train_layer_ids = list(range(18, 26))

for name, param in model.named_parameters():
    if (
        any(name.startswith(f"model.layers.{i}") for i in train_layer_ids)
        or name.startswith("model.norm")
        or name.startswith("lm_head")
    ):
        param.requires_grad = True
    else:
        param.requires_grad = False

"""
print("학습되는 Gemma:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(" -", name)
"""

# DuckDuckGo 검색기 초기화
search_tool = DuckDuckGoSearchRun()

# 토크나이즈 함수 정의
def tokenize_batch(batch):
    prompts = []
    full_texts = []

    for question, answer in zip(batch["question"], batch["real_answer"]):
        context = get_cached_or_search(question)
        prompt = f"""
반드시 한국어로 대답해! 너는 코드 리뷰 전문가야. 아래 검색 결과와 코드 조각을 참고해서 코드에 어떤 문제가 있고 어떻게 개선할 수 있는지 조언을 줘.
추측하지 말고, 검색 결과와 문맥 안의 정보만 사용해. 실제 코드 리뷰처럼 구조적인 피드백을 줘.

[검색 결과]
{context}

[질문]
{question}

[답변]
""".strip()

        full_text = prompt + "\n" + answer
        full_texts.append(full_text)
        prompts.append(prompt)

    tokenized = tokenizer(
        full_texts,
        truncation=True,
        padding="max_length",
        max_length=4096,
        return_tensors="pt"
    )

    labels = tokenized["input_ids"].clone()

    for i, prompt in enumerate(prompts):
        prompt_ids = tokenizer(prompt, truncation=True, max_length=4096)["input_ids"]
        prompt_len = len(prompt_ids)

        # 프롬프트 부분은 loss 계산 제외
        labels[i][:prompt_len] = -100

    tokenized["labels"] = labels
    return {k: v.tolist() for k, v in tokenized.items()}

def tokenize_batch_seq2seq(batch):
    inputs = []
    targets = []

    for question, answer in zip(batch["question"], batch["real_answer"]):
        context = get_cached_or_search(question)

        prompt = f"""
반드시 한국어로 대답해! 너는 코드 리뷰 전문가야. 아래 검색 결과와 코드 조각을 참고해서 코드에 어떤 문제가 있고 어떻게 개선할 수 있는지 조언을 줘.
추측하지 말고, 검색 결과와 문맥 안의 정보만 사용해. 실제 코드 리뷰처럼 구조적인 피드백을 줘.

[검색 결과]
{context}

[질문]
{question}

[답변]
""".strip()

        inputs.append(prompt)
        targets.append(answer)

    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=4096,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            truncation=True,
            padding="max_length",
            max_length=4096,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# 5. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./gemma_rag_gpt4o_distilled",
    per_device_train_batch_size=1,
    learning_rate=5e-6,
    num_train_epochs=10,
    fp16=True,
    logging_steps=10,
    save_strategy="no"
)

from datasets import load_dataset
import random
from transformers import TrainerCallback
from langchain_openai import ChatOpenAI

# 3. JSON 데이터 로드 및 분할
split = load_dataset("json", data_files="generated_data.json")["train"] \
    .shuffle(seed=42) \
    .select(range(980)) \
    .train_test_split(test_size=0.2)

train_dataset = split["train"]
test_dataset = split["test"]

# 4. 토크나이즈 + 기존 컬럼 제거
tokenized_train = train_dataset.map(tokenize_batch, batched=True, remove_columns=train_dataset.column_names, keep_in_memory=False)
tokenized_test = test_dataset.map(tokenize_batch, batched=True, remove_columns=test_dataset.column_names, keep_in_memory=False)
tokenized_train.set_format(type="torch")
tokenized_test.set_format(type="torch")

print(tokenized_train[0])

# 6. Epoch 끝날 때마다 저장
class EpochEvalCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # model.save_pretrained("./checkpoints/gemma_gpt")
        # tokenizer.save_pretrained("./checkpoints/gemma_gpt")
        print("Epoch End")


# 7. Trainer 정의 및 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    callbacks=[EpochEvalCallback()]
)

evaluate_on_test_samples(model, tokenizer, test_dataset)
trainer.train()
evaluate_on_test_samples(model, tokenizer, test_dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/784 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

{'input_ids': tensor([     0,      0,      0,  ..., 198766, 204551, 235265]), 'attention_mask': tensor([0, 0, 0,  ..., 1, 1, 1]), 'labels': tensor([  -100,   -100,   -100,  ..., 198766, 204551, 235265])}

--- 테스트 데이터셋 일부 평가 ---

[1] 질문: ```python
numbers = [1, 2, 3, 4, 5]
squared_numbers = []
for i in range(len(numbers)):
    squared_numbers.append(numbers[i] * numbers[i])

result = []
for num in squared_numbers:
    if num % 2 == 0:
        result.append(num)
        
output = ''
for r in result:
    output += str(r) + ', '
if output.endswith(', '):
    output = output[:-2]
    
final_result = 'Even squared numbers: ' + output
print(final_result)
```

정답:
이 코드 조각에 대한 리뷰를 진행하겠습니다. 이 코드는 주어진 숫자 목록에서 제곱한 후, 짝수인 제곱수를 출력하는 작업을 수행합니다. 각 부분을 단계적으로 살펴보고 개선 방안을 제안하겠습니다.

### 1. 변수 초기화 및 리스트 컴프리헨션 사용
- **문제점**: 첫 번째 `for` 루프에서는 `squared_numbers` 리스트에 제곱 값을 저장하고 있습니다. 이 과정은 리스트 컴프리헨션을 사용하여 더 간결하게 표현할 수 있습니다.
- **개선 방안**:
  ```python
  squared_numbers = [x * x for x in numbers]
  ```

### 2. 조건 필

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


...

Epoch End
Epoch End
Epoch End
Epoch End
Epoch End
Epoch End
Epoch End
Epoch End
Epoch End
Epoch End

--- 테스트 데이터셋 일부 평가 ---

[1] 질문: ```python
numbers = [1, 2, 3, 4, 5]
squared_numbers = []
for i in range(len(numbers)):
    squared_numbers.append(numbers[i] * numbers[i])

result = []
for num in squared_numbers:
    if num % 2 == 0:
        result.append(num)
        
output = ''
for r in result:
    output += str(r) + ', '
if output.endswith(', '):
    output = output[:-2]
    
final_result = 'Even squared numbers: ' + output
print(final_result)
```

정답:
이 코드 조각에 대한 리뷰를 진행하겠습니다. 이 코드는 주어진 숫자 목록에서 제곱한 후, 짝수인 제곱수를 출력하는 작업을 수행합니다. 각 부분을 단계적으로 살펴보고 개선 방안을 제안하겠습니다.

### 1. 변수 초기화 및 리스트 컴프리헨션 사용
- **문제점**: 첫 번째 `for` 루프에서는 `squared_numbers` 리스트에 제곱 값을 저장하고 있습니다. 이 과정은 리스트 컴프리헨션을 사용하여 더 간결하게 표현할 수 있습니다.
- **개선 방안**:
  ```python
  squared_numbers = [x * x for x in numbers]
  ```

### 2. 조건 필터링과 리스트 컴프리헨션
- **문제점**: 두 번째 `for` 루프 사용 시 조건에 맞는 값을 `result` 리스트에 추가하고 있습니다. 리스트 컴프리헨션을 활용하면 더 명확하고 간결

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[2] 질문: ```python
numbers = [1, 2, 3, 4, 5]
squared_numbers = []
for number in numbers:
    square = number * number
    squared_numbers.append(square)
print(squared_numbers)

words = ["apple", "banana", "cherry"]
found = False
for word in words:
    if word == "banana":
        found = True
if found:
    print("Banana is in the list")
else:
    print("Banana is not in the list")

total = 0
num_list = [1, 2, 3, 4, 5]
for i in range(len(num_list)):
    total += num_list[i]
print(total)

x = 10
y = 25
if x > y:
    minimum = y
else:
    minimum = x
print("The minimum value is", minimum)

even_numbers = []
for i in range(1, 11):
    if i % 2 == 0:
        even_numbers.append(i)
print(even_numbers)

words = ["hello", "world", "python"]
uppercase_words = []
for word in words:
    uppercase_word = word.upper()
    uppercase_words.append(uppercase_word)
print(uppercase_words)
```

정답:
이 코드는 기능적으로 올바르게 작동하긴 하지만, Python의 관용적이고 효율적인 코딩 스타일에 맞추어 개선할 수 있습니다. 아래에 코드의 각 부분에 대한 구체적인 피드백과 개선점을 제공하겠습니다

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[3] 질문: ```python
import random

def complex_function(data):
    if not isinstance(data, list):
        return "Input should be a list"

    result = 0
    for i in range(len(data)):
        if i % 2 == 0:
            result += data[i] ** 2
        elif i % 3 == 0:
            result -= data[i] / 2

    random.shuffle(data)
    extra_value = random.choice(data)
    
    if result % 5 == 0 or result % 7 == 0:
        result *= extra_value

    try:
        final_result = result / len(set(data))
    except ZeroDivisionError:
        final_result = float('inf')

    if final_result < 0:
        return abs(final_result)
    
    sorted_data = sorted(data)
    median = sorted_data[len(sorted_data) // 2]
    
    output = (final_result, median)
    return output

test_data = [random.randint(1, 100) for _ in range(20)]
output = complex_function(test_data)
print(f"Output: {output}")
```

정답:
이 코드는 다양한 기능을 수행하지만 몇 가지 문제가 있고 개선할 수 있는 부분이 있습니다. 각 문제를 설명하고 해결책을 제안하겠습니다.

### 문제 및 개선점

1. **코드의 명확성